**Building-Usage Data Preprocessing**

Data Source:

  ./[Metropolitan_Area|Province]/[City|County|District(gu)]/*.zip: http://openapi.nsdi.go.kr/nsdi/ (용도별건물정보 (Building information by use))
  

Input:

  ./[Metropolitan_Area|Province]/[City|County|District(gu)]/*.zip


Output:

  ./building_use.ftr

In [1]:
import glob
import geopandas as gpd
import pandas as pd
import os
from tqdm import notebook
from collections import OrderedDict
from multiprocessing import Pool as ProcessPool
from datetime import datetime as dt
import re

In [2]:
do_unzip = False

# unzip all files
zip_paths = glob.glob('./*/*/*.zip')
if __name__ == '__main__' and do_unzip:
  for z_path in notebook.tqdm(zip_paths):
    dir_name = os.path.dirname(z_path) + '/' + z_path[-12:-4] # extraction dir name: data creation date
    !unzip -nq -I EUC-KR {z_path} -d {dir_name}

In [3]:
kor_crs = "+proj=tmerc +lat_0=38 +lon_0=127.0028902777778 +k=1 +x_0=200000 +y_0=500000 "\
            "+ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43"

def load_subway():
  net_nodes = pd.read_json('../subway/network_nodes.json')
  net_nodes = gpd.GeoDataFrame(
    net_nodes, geometry=gpd.points_from_xy(x=net_nodes.lng, y=net_nodes.lat), crs='EPSG:4326'
  )
  net_nodes.to_crs(kor_crs, inplace=True)
  net_nodes = net_nodes[['station_id', 'geometry']]
  net_nodes.set_index('station_id', inplace=True)
  return net_nodes

In [4]:
col_map = {
  'A1': 'building_ID',
  'A19': 'total_area',
  'A25': 'usage_general',
  'A27': 'usage_specific',
  'A29': 'usage_class',
  'A34': 'start_date', # Date of approval for use
  'A35': 'ref_date', # Data creation reference date
  'geometry': 'geometry',
}

save_cols = {
  'building_ID': str, 
  'total_area': float, 
  'usage_general': str, 
  'usage_specific': str, 
  'usage_class': str, 
  'start_date': str, 
  'ref_date': str, 
  'adjacent_stations': set
}

usage_col = 'usage_specific'

selected_cols = ['building_ID', 'total_area', usage_col, 'start_date', 'ref_date', 'adjacent_stations']

net_nodes = load_subway()

# date boundaries
i_min_date = 20170101
i_max_date = 20201231
min_date = dt.strptime(str(i_min_date), '%Y%m%d').date()
max_date = dt.strptime(str(i_max_date), '%Y%m%d').date()
min_valid_date = pd.Timestamp.min.date()
max_valid_date = pd.Timestamp.max.date()

date_regexp = re.compile(r'\d{4}\-(0[1-9]|1[012])\-(0[1-9]|[12][0-9]|3[01])')

def read_df(shp_path: str):
  gdf: gpd.GeoDataFrame = gpd.read_file(shp_path, encoding="euc-kr")[col_map.keys()]
  gdf.rename(columns=col_map, inplace=True)
  gdf = gdf[(gdf['total_area'].notnull())]
  gdf.drop_duplicates(subset='building_ID', inplace=True, ignore_index=True)
  gdf.crs = 'epsg:5174'
  gdf.to_crs(kor_crs, inplace=True)
  # keep significant buildings only (500m or less from at least one subway station)
  gdf: gpd.GeoDataFrame = gdf.sjoin_nearest(net_nodes, how='left', max_distance=501, distance_col='nearest_dist')
  gdf.drop_duplicates(subset='building_ID', inplace=True, ignore_index=True)
  gdf = gdf[(gdf['nearest_dist'].notna()) & (gdf['nearest_dist'] <= 500)]
  # map buildings to their adjacent stations
  distances: pd.DataFrame = gdf.geometry.apply(lambda g: net_nodes.distance(g))
  gdf['adjacent_stations'] = distances.apply(lambda dist: set(dist[dist<=500].index), axis=1)
  return gdf[save_cols.keys()]

def str_to_date(date: str):
  # avoid parsing irrelevant dates
  i_date = int(date.replace('-', ''))
  if i_date < i_min_date:
    return min_valid_date
  elif i_date > i_max_date:
    return max_valid_date
  # parse date
  return pd.to_datetime(date, format='%Y-%m-%d').date()

def process_shp(shp_path: str):
  json_path = shp_path[:-3] + 'json'
  if os.path.exists(json_path):
    df: pd.DataFrame = pd.read_json(json_path, dtype=save_cols)
  else:
    df = read_df(shp_path)
    df.to_json(json_path)
  assert df['building_ID'].nunique() == len(df)
  df['start_date'] = [date if date_regexp.search(date) else None for date in df['start_date']]
  df = df[(df['start_date'].notnull()) & (df['ref_date'].notnull())]
  df['start_date'] = df['start_date'].apply(str_to_date)
  df['ref_date'] = df['ref_date'].apply(str_to_date)
  # filter by date (must be active in the relevant time period range)
  df = df[(df['start_date'] <= max_date) & (df['ref_date'] >= min_date)]
  return df[selected_cols]

if __name__ == '__main__':
  workers = 3 # number of processes used to preprocess all files
  
  shp_paths = glob.glob('./*/*/*/*.shp')
  assert len(zip_paths) == len(shp_paths)
  with ProcessPool(workers) as pool:
    try:
      df: pd.DataFrame = pd.concat(list(notebook.tqdm(pool.imap_unordered(process_shp, shp_paths), total=len(shp_paths))), ignore_index=True, copy=False)
    except Exception as e:
      print(f'EXCEPTION: {e}. Terminating pool...')
      pool.terminate()
      print('Pool terminated')
      raise Exception(e)
# keep the most recent entry for each (building_ID, usage)-pair only
df.sort_values(by='ref_date', ascending=False, inplace=True, na_position='last', ignore_index=True)
df.drop_duplicates(subset=['building_ID', usage_col], keep='first', inplace=True, ignore_index=True)
print(df.shape)
df.head(10)

  0%|          | 0/634 [00:00<?, ?it/s]

(395204, 6)


,building_ID,total_area,usage_specific,start_date,ref_date,adjacent_stations
0,2002203640844689275400000000,627.51,다세대주택,1677-09-21,2262-04-11,[515]
1,1993204644014517550900000000,141.56,다가구주택,1677-09-21,2262-04-11,[103]
2,1974204850594517422900000000,99.34,단독주택,1677-09-21,2262-04-11,[103]
3,1969204843704517319400000000,233.29,단독주택,1677-09-21,2262-04-11,[103]
4,1991204852454517222500000000,235.34,단독주택,1677-09-21,2262-04-11,[103]
5,2003204862424517342600000000,659.69,다세대주택,1677-09-21,2262-04-11,[103]
6,1990204857704517495800000000,160.27,다가구주택,1677-09-21,2262-04-11,[103]
7,1989204867434517563400000000,166.12,단독주택,1677-09-21,2262-04-11,[103]
8,1971204667004517379500000000,145.66,단독주택,1677-09-21,2262-04-11,[103]
9,1985204656494517408600000000,39.10,단독주택,1677-09-21,2262-04-11,[103]


In [5]:
def process_bu(df: pd.DataFrame):
  keys = OrderedDict()
  data = {usage:[] for usage in df[usage_col].unique()}
  for _, area, usage, start_date, ref_date, adj_sts in notebook.tqdm(df.itertuples(index=False, name=None), total=len(df)):
    for date in pd.date_range(start=max(start_date, min_date), end=min(ref_date, max_date)):
      date = date.date()
      for station_id in adj_sts:
        key = (date, station_id)
        if key in keys:
          data[usage][keys[key]] += area
        else:
          keys[key] = len(data[usage])
          for usage_t, l in data.items():
            l.append(area if usage == usage_t else 0)
  data = pd.DataFrame(data, index=pd.MultiIndex.from_tuples(keys.keys(), names=['date', 'station_id']), copy=False)
  return data

df = process_bu(df)
print(df.shape)
df.sort_index(inplace=True)
df.reset_index(inplace=True, drop=False)
df.to_feather('building_use.ftr')
df.head(15)

  0%|          | 0/395204 [00:00<?, ?it/s]

(755337, 343)


/tmp/ipykernel_29842/3008927128.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df.reset_index(inplace=True, drop=False)


,date,station_id,다세대주택,다가구주택,단독주택,소매점,제1종근린생활시설,기타제1종근생활시설,기타사무소,기타제2종근생활시설,...,양잠,경마장,종합여객시설,군사시설,공항시설,제실,식물원,요양소,극장,어린이회관
0,2017-01-01,0,135259.340,50472.300,77525.5340,10182.528,10631.240,19202.350,5757.31,884.270,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-01,1,86616.902,34882.865,214201.2530,7697.050,11763.805,80914.241,935.28,8372.070,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-01,2,34397.665,10437.790,26222.5650,11515.400,19133.540,14298.050,451.41,10360.820,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-01,3,16672.910,48365.460,29357.4200,6267.780,3184.660,9400.430,1749.62,5031.060,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-01,4,2970.430,0.000,0.0000,2589.880,10287.140,1970.720,1406.40,7342.130,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2017-01-01,5,229991.779,121432.610,124920.2090,29010.820,22452.365,28021.470,5022.85,2093.990,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2017-01-01,6,84283.420,47108.910,56199.2500,19616.020,8393.640,24464.180,505.98,7675.210,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2017-01-01,7,14695.170,200110.648,14209.3400,3856.940,3142.530,4669.710,1331.97,1112.960,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2017-01-01,8,3993.690,898.010,2022.2200,2258.300,0.000,111.980,108.40,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2017-01-01,9,87821.295,15524.756,54250.4065,13214.220,10970.175,11372.430,391.50,12436.898,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
